# 📊 Proyecto Avanzado: Análisis de Ventas con Pandas + Matplotlib (PLUS)
Este cuaderno guía un análisis **end-to-end**: carga/validación de datos, EDA, limpieza, *feature engineering*, KPIs,
visualizaciones y analítica de clientes (RFM y cohortes). Incluye **tareas** con **soluciones ocultas**.

**Objetivos:**
- Practicar carga y validación de datos con `pandas`.
- Crear KPIs de negocio (ingresos, ticket medio, devoluciones, top productos/clientes).
- Usar `groupby`, *pivot tables*, re-muestreo por mes y gráficos con **Matplotlib**.
- Analizar clientes con **RFM** y **cohortes**.

**Entregables:** CSV con resultados clave, 3 gráficos y un breve texto con hallazgos.

**ACLARACIONES:**
EDA (Análisis Exploratorio de Datos) te dice qué hay y cómo están los datos; RFM (Recencia -días de la última compra -, Frecuencia y Valor Monetario) te dice qué clientes son más valiosos/activos; cohortes (Grupo de clientes definido por su momento de alta) te dice cómo evoluciona la retención en el tiempo.

## 0) Preparación: librerías y helpers

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from datetime import datetime
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 120)

## 1) Dataset: generar o cargar
Si existe `ventas_simuladas.csv`, lo cargamos. Si no, generamos uno reproducible.

In [ ]:
def generar_dataset_csv(ruta='ventas_simuladas.csv', seed=42):
    rng = np.random.default_rng(seed)
    dates = pd.date_range('2023-01-01', '2024-12-31', freq='D')
    products = [
        ('Teclado','Periféricos'),('Ratón','Periféricos'),('Monitor','Pantallas'),
        ('Portátil','Ordenadores'),('Disco SSD','Almacenamiento'),('Impresora','Periféricos'),
        ('Auriculares','Periféricos'),('Base refrigeradora','Periféricos'),('Webcam','Periféricos')
    ]
    rows = []
    order_id = 1
    for d in dates:
        for _ in range(rng.integers(5, 20)):
            prod, cat = products[rng.integers(0, len(products))]
            units = int(rng.integers(1, 5))
            base = 50 if cat=='Periféricos' else (300 if cat in ['Ordenadores','Pantallas'] else 100)
            price = float(rng.normal(base, base*0.35))
            price = max(price, 5)
            returned = rng.random() < 0.05
            rows.append((order_id, d, rng.integers(1, 801), prod, cat, units, round(price,2), returned))
            order_id += 1
    df = pd.DataFrame(rows, columns=['order_id','order_date','customer_id','product','category','units','unit_price','returned'])
    df.to_csv(ruta, index=False)
    return ruta

csv_path = Path('ventas_simuladas.csv')
if not csv_path.exists():
    generar_dataset_csv(csv_path)
df = pd.read_csv(csv_path, parse_dates=['order_date'])
df.head()

## 2) Validación y limpieza mínima

In [ ]:
# Tipos y nulos
display(df.info())
nulos = df.isna().sum()
nulos

In [ ]:
# Reglas de negocio simples
assert (df['units']>=1).all(), 'Units debe ser >= 1'
assert (df['unit_price']>0).all(), 'unit_price debe ser > 0'
df['returned'] = df['returned'].astype(bool)
df['category'] = df['category'].astype('category')
df['product'] = df['product'].astype('category')
df['order_date'] = pd.to_datetime(df['order_date'])
df['revenue'] = df['units'] * df['unit_price']
df.head()

## 3) EDA rápida

In [ ]:
df.describe(include='all')

In [ ]:
df.groupby('category')['revenue'].sum().sort_values(ascending=False).head(10)

### Tarea 1: ¿Cuál es el rango de fechas y el nº de pedidos únicos?

<details><summary><b>Ver solución</b></summary>

```python
df['order_date'].min(), df['order_date'].max(), df['order_id'].nunique()
```
</details>

## 4) KPIs básicos

In [ ]:
# KPI 1: Ingresos por mes
df['month'] = df['order_date'].dt.to_period('M').dt.to_timestamp()
ingresos_mes = df.groupby('month')['revenue'].sum().reset_index()
ingresos_mes.head()

In [ ]:
# KPI 2: Ticket medio por pedido
ticket_medio = df.groupby('order_id')['revenue'].sum().mean()
ticket_medio

In [ ]:
# KPI 3: Top 10 productos por ingresos
top_prod = df.groupby('product')['revenue'].sum().sort_values(ascending=False).head(10)
top_prod

In [ ]:
# KPI 4: Tasa de devolución
tasa_dev = df['returned'].mean()
tasa_dev

## 5) Visualizaciones con Matplotlib (una figura por gráfico)

In [ ]:
# Ingresos por mes (línea)
ingresos_mes.plot(x='month', y='revenue', kind='line', title='Ingresos por mes')
plt.show()

In [ ]:
# Ingresos por categoría (barra)
df.groupby('category')['revenue'].sum().sort_values().plot(kind='bar', title='Ingresos por categoría')
plt.show()

In [ ]:
# Top 10 productos (barra)
df.groupby('product')['revenue'].sum().sort_values(ascending=False).head(10).plot(kind='bar', title='Top 10 productos')
plt.show()

### Tarea 2: Representa la evolución de la **tasa de devolución** por mes

<details><summary><b>Ver solución</b></summary>

```python
dev_mes = df.groupby('month')['returned'].mean().reset_index()
dev_mes.plot(x='month', y='returned', kind='line', title='Tasa de devolución por mes')
plt.show()
```
</details>

## 6) Analítica de clientes: **RFM**

In [ ]:
# Calcula Recency (días desde última compra), Frequency (nº pedidos), Monetary (ingresos)
ref_date = df['order_date'].max() + pd.Timedelta(days=1)
agg = df.groupby('customer_id').agg(
    last_purchase=('order_date','max'),
    frequency=('order_id','nunique'),
    monetary=('revenue','sum')
).reset_index()
agg['recency'] = (ref_date - agg['last_purchase']).dt.days
agg.head()

In [ ]:
# Discretización por quintiles para r, f, m → scores 1–5
def quintil_scorer(s, reverse=False):
    q = s.rank(pct=True)
    sc = (q*5).apply(np.ceil).astype(int)
    sc = sc.clip(1,5)
    return 6 - sc if reverse else sc
agg['R'] = quintil_scorer(agg['recency'], reverse=True)  # menor recency → mejor
agg['F'] = quintil_scorer(agg['frequency'])
agg['M'] = quintil_scorer(agg['monetary'])
agg['RFM'] = agg['R'].astype(str)+agg['F'].astype(str)+agg['M'].astype(str)
agg.sort_values('monetary', ascending=False).head(10)

### Tarea 3: ¿Qué porcentaje de ingresos aportan los **top 10%** de clientes por `monetary`?

<details><summary><b>Ver solución</b></summary>

```python
n = max(1, int(len(agg)*0.10))
top = agg.sort_values('monetary', ascending=False).head(n)
pct = top['monetary'].sum() / agg['monetary'].sum()
pct
```
</details>

## 7) Analítica de **cohortes** (retención mensual)
Definimos la cohorte por **mes de primera compra** y construimos una tabla de retención.

In [ ]:
df_small = df[['customer_id','order_id','order_date','revenue']].copy()
df_small['first_month'] = df_small.groupby('customer_id')['order_date'].transform('min').dt.to_period('M').dt.to_timestamp()
df_small['month'] = df_small['order_date'].dt.to_period('M').dt.to_timestamp()
df_small['cohort_index'] = ((df_small['month'].dt.year - df_small['first_month'].dt.year)*12 +
                             (df_small['month'].dt.month - df_small['first_month'].dt.month))
retencion = (df_small
    .groupby(['first_month','cohort_index'])['customer_id']
    .nunique()
    .reset_index())
base = retencion[retencion['cohort_index']==0][['first_month','customer_id']].rename(columns={'customer_id':'n0'})
tabla = retencion.merge(base, on='first_month')
tabla['ret'] = tabla['customer_id']/tabla['n0']
cohort_pivot = tabla.pivot(index='first_month', columns='cohort_index', values='ret').fillna(0)
cohort_pivot.head()

In [ ]:
# Visualización rápida de cohortes (matriz)
plt.figure()
plt.imshow(cohort_pivot.values, aspect='auto')
plt.title('Retención por cohortes (proporción)')
plt.xlabel('Meses desde primera compra')
plt.ylabel('Cohorte (mes inicio)')
plt.colorbar()
plt.show()

### Tarea 4: Calcular el **LTV** aproximado por cohorte como suma de ingresos medios por cliente a 3 meses

<details><summary><b>Ver solución (idea)</b></summary>

```python
ingresos_cliente_mes = (df_small
    .groupby(['first_month','month'])
    .agg(ingresos=('revenue','sum'), clientes=('customer_id','nunique'))
    .reset_index())
ingresos_cliente_mes['cohort_index'] = ((ingresos_cliente_mes['month'].dt.year - ingresos_cliente_mes['first_month'].dt.year)*12 +
                                        (ingresos_cliente_mes['month'].dt.month - ingresos_cliente_mes['first_month'].dt.month))
ing_cli = ingresos_cliente_mes.pivot_table(index='first_month', columns='cohort_index', values='ingresos')
cli = ingresos_cliente_mes.pivot_table(index='first_month', columns='cohort_index', values='clientes')
arpu = ing_cli/cli
ltv3 = arpu[[0,1,2]].sum(axis=1)
ltv3.head()
```
</details>

## 8) Export de resultados

In [ ]:
ingresos_mes.to_csv('kpi_ingresos_mensuales.csv', index=False)
agg[['customer_id','R','F','M','RFM','recency','frequency','monetary']].to_csv('rfm_clientes.csv', index=False)
cohort_pivot.to_csv('retencion_cohortes.csv')
print('✅ Exportados: kpi_ingresos_mensuales.csv, rfm_clientes.csv, retencion_cohortes.csv')

## 9) Retos extra (para nota)
1) **Anomalías**: detecta días con ingresos > μ+3σ y coméntalos.
2) **Segmentación**: clasifica clientes en 4 grupos (p.ej. KMeans sobre (recency, frequency, monetary)).
3) **AB testing simulado**: crea una bandera de campaña y compara ticket medio con test estadístico.
4) **Performance**: compara tiempo de `groupby` tras convertir `product` y `category` a `category`.

## 10) Rúbrica (10 puntos)
- Carga/validación y limpieza básica (2)
- KPIs correctos y bien interpretados (2)
- Gráficos claros (3) con conclusiones (2)
- Analítica de clientes (RFM/cohortes) (3)
- Export de resultados (1)
- Calidad del cuaderno (narrativa, orden, reproducibilidad) (+1 bonus)